# Process U-Net data and save on disk

Use this script to save processed version of the samples in the dataset, their annotations, or their predictions on disk.

Author: Prisca Dotti  
Last modified: 02.11.2023

In [43]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import logging
import os
import random
import imageio

import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from torch import nn
from torch.utils.data import DataLoader

from config import TrainingConfig, config
from data.data_processing_tools import preds_dict_to_mask, process_raw_predictions
from utils.in_out_tools import write_colored_events_videos_on_disk
from utils.training_inference_tools import do_inference
from utils.training_script_utils import init_dataset, init_model
from utils.visualization_tools import (
    add_colored_segmentation_to_video,
    get_annotations_contour,
)

logger = logging.getLogger(__name__)

### Load movies and annotations

In [45]:
# movie_ids = ["05", "10", "15", "20", "25", "32", "34", "40", "45"]
# movie_ids = ["05", "34"]
movie_ids = ["05"]

In [46]:
training_name = "final_model"
config_filename = "config_final_model.ini"
load_epoch = 100000

# Initialize general parameters
params = TrainingConfig(
    training_config_file=os.path.join("config_files", config_filename)
)
params.run_name = training_name
# params.set_device(device="auto")
params.set_device(device="cpu")
model_filename = f"network_{load_epoch:06d}.pth"

# Output directory
out_dir = os.path.join(
    config.basedir, "evaluation", "processed_movies_script", training_name
)
os.makedirs(out_dir, exist_ok=True)

[22:21:45] [  INFO  ] [   config   ] <290 > -- Loading C:\Users\prisc\Code\sparks_project\config_files\config_final_model.ini


In [47]:
# Create dataset
dataset = init_dataset(
    params=params,
    sample_ids=movie_ids,
    apply_data_augmentation=False,
    print_dataset_info=True,
    load_instances=True,
)

# Create a dataloader
dataset_loader = DataLoader(
    dataset,
    batch_size=params.inference_batch_size,
    shuffle=False,
    num_workers=params.num_workers,
    pin_memory=params.pin_memory,
)

xs = dataset.get_movies()
ys = dataset.get_labels()
ys_instances = dataset.get_instances()

[22:21:56] [  INFO  ] [utils.training_script_utils] <137 > -- Samples in dataset: 9


### Configure and load trained model

In [6]:
# Configure model architecture
network = init_model(params=params)

# Move the model to the GPU if available
if params.device.type != "cpu":
    network = nn.DataParallel(network).to(params.device, non_blocking=True)
    # cudnn.benchmark = True

# Load model weights
models_relative_path = os.path.join(
    "models", "saved_models", params.run_name, model_filename
)
model_dir = os.path.realpath(os.path.join(config.basedir, models_relative_path))

In [7]:
# Load the model state dictionary
print(f"Loading trained model '{params.run_name}' at epoch {load_epoch}...")
try:
    network.load_state_dict(torch.load(model_dir, map_location=params.device))
except RuntimeError as e:
    if "module" in str(e):
        # The error message contains "module," so handle the DataParallel loading
        print(
            "Failed to load the model, as it was trained with DataParallel. Wrapping it in DataParallel and retrying..."
        )
        # Get current device of the object (model)
        temp_device = next(iter(network.parameters())).device

        network = nn.DataParallel(network)
        network.load_state_dict(torch.load(model_dir, map_location=params.device))

        print("Network should be on CPU, removing DataParallel wrapper...")
        network = network.module.to(temp_device)
    else:
        # Handle other exceptions or re-raise the exception if it's unrelated
        raise

Loading trained model 'final_model' at epoch 100000...
Failed to load the model, as it was trained with DataParallel. Wrapping it in DataParallel and retrying...
Network should be on CPU, removing DataParallel wrapper...


### Get U-Net's predictions

In [8]:
# get U-Net's raw predictions
network.eval()
raw_preds_inference = do_inference(
    network=network,
    params=params,
    dataloader=dataset_loader,
    device=params.device,
    compute_loss=False,
    inference_types=[params.inference],
)

# Remove middle dictionary from raw_preds
raw_preds = {idx: pred[params.inference] for idx, pred in raw_preds_inference.items()}

In [9]:
# get processed predictions
final_segmentation_dict = {}
final_instances_dict = {}
sparks_coords_dict = {}

for i in range(len(movie_ids)):
    # transform raw predictions into a dictionary
    raw_preds_dict = {
        event_type: raw_preds[i][event_label]
        for event_type, event_label in config.classes_dict.items()
        if event_type in config.event_types
    }

    pred_instances, pred_segmentation, sparks_coords = process_raw_predictions(
        raw_preds_dict=raw_preds_dict,
        input_movie=xs[i],
        training_mode=False,
        debug=False,
    )

    final_segmentation_dict[movie_ids[i]] = pred_segmentation
    final_instances_dict[movie_ids[i]] = pred_instances
    sparks_coords_dict[movie_ids[i]] = sparks_coords

## Paste colored segmentation on original movie
Use this section to add colored annotations and predictions on given dataset movies.  
Used to generate examples for midterm exams.

In [11]:
segmentation_transparency = 70
instances_transparency = 90


for i, movie_id in enumerate(movie_ids):
    print("Processing video", movie_id, "...")
    # Get processed predictions as numpy arrays of integers
    pred_segmentation = preds_dict_to_mask(final_segmentation_dict[movie_id])
    pred_instances = np.array(sum(final_instances_dict[movie_id].values()))

    base_fn = f"{training_name}_{load_epoch:06d}_{movie_id}"

    print("Saving predicted segmentation masks on disk...")
    write_colored_events_videos_on_disk(
        movie=xs[i],
        events_mask=pred_segmentation,
        out_dir=out_dir,
        movie_fn=f"{base_fn}_colored_classes_preds",
        transparency=segmentation_transparency,
        ignore_frames=0,
        white_bg=False,
        instances=False,
    )

    print("Saving labelled segmentation masks on disk...")
    write_colored_events_videos_on_disk(
        movie=xs[i],
        events_mask=ys[i],
        out_dir=out_dir,
        movie_fn=f"{base_fn}_colored_classes_labels",
        transparency=segmentation_transparency,
        ignore_frames=0,
        white_bg=False,
        instances=False,
    )

    # Add transparent instances masks to input movie and save on disk

    print("Saving predicted instances masks on disk...")
    write_colored_events_videos_on_disk(
        movie=xs[i],
        events_mask=pred_instances,
        out_dir=out_dir,
        movie_fn=f"{base_fn}_colored_instances_preds",
        transparency=instances_transparency,
        ignore_frames=0,
        white_bg=False,
        instances=True,
    )

    print("Saving labelled instances masks on disk...")
    write_colored_events_videos_on_disk(
        movie=xs[i],
        events_mask=ys_instances[i],
        out_dir=out_dir,
        movie_fn=f"{base_fn}_colored_instances_labels",
        transparency=segmentation_transparency,
        ignore_frames=0,
        white_bg=False,
        instances=True,
    )

    print(
        "Saving movies where annotations are denoted by border and preds are transparent..."
    )
    write_colored_events_videos_on_disk(
        movie=xs[i],
        events_mask=pred_segmentation,
        out_dir=out_dir,
        movie_fn=f"{base_fn}_colored_preds_and_labels",
        transparency=segmentation_transparency,
        ignore_frames=params.ignore_frames_loss,
        white_bg=False,
        instances=False,
        label_mask=ys[i],
    )

    print("Saving predicted segmentation masks on white background...")
    write_colored_events_videos_on_disk(
        movie=xs[i],
        events_mask=pred_segmentation,
        out_dir=out_dir,
        movie_fn=f"{base_fn}_colored_preds_white_bg",
        transparency=segmentation_transparency,
        ignore_frames=params.ignore_frames_loss,
        white_bg=True,
    )

    print("Saving labelled segmentation masks on white background...")
    write_colored_events_videos_on_disk(
        movie=xs[i],
        events_mask=ys[i],
        out_dir=out_dir,
        movie_fn=f"{base_fn}_colored_labels_white_bg",
        transparency=segmentation_transparency,
        ignore_frames=params.ignore_frames_loss,
        white_bg=True,
    )

Processing video 05 ...
Saving predicted segmentation masks on disk...
Saving labelled segmentation masks on disk...
Saving predicted instances masks on disk...
Saving labelled instances masks on disk...
Saving movies where annotations are denoted by border and preds are transparent...
Saving predicted segmentation masks on white background...
Saving labelled segmentation masks on white background...


### Save composed movies with segmentation masks and instance masks

In [ ]:
for i, movie_id in enumerate(movie_ids):
    # Get processed predictions as numpy arrays of integers
    pred_segmentation = preds_dict_to_mask(final_segmentation_dict[movie_id])
    pred_instances = np.array(sum(final_instances_dict[movie_id].values()))

    segmentation_transparency = 70
    instances_transparency = 90
    movie_rgb = np.copy(255 * (xs[i] / xs[i].max()))
    movie_rgb = [Image.fromarray(frame).convert("RGB") for frame in movie_rgb]

    classes_dict = {  # Dataset parameters
        "sparks": {"nb": 1, "color": [178, 255, 102]},  # Green
        "puffs": {"nb": 3, "color": [255, 102, 102]},  # Red
        "waves": {"nb": 2, "color": [178, 102, 255]},  # Purple
        "ignore": {"nb": 4, "color": [224, 224, 224]},  # Gray
    }

    # Create movie with colored segmentation masks
    segmentation_rgb = np.copy(255 * (xs[i] / xs[i].max()))
    segmentation_rgb = [
        Image.fromarray(frame).convert("RGB") for frame in segmentation_rgb
    ]

    for class_info in classes_dict.values():
        class_nb = class_info["nb"]
        color = class_info["color"]

        # Add colored predicted segmentation mask
        if class_nb in pred_segmentation:
            binary_preds = pred_segmentation == class_nb
            segmentation_rgb = add_colored_segmentation_to_video(
                segmentation=binary_preds,
                video=segmentation_rgb,
                color=color,
                transparency=segmentation_transparency,
            )

        # Add annotated label contours
        label_contours = get_annotations_contour(annotations=ys[i], contour_val=2)
        if class_nb in ys[i]:
            binary_labels = label_contours == class_nb
            segmentation_rgb = add_colored_segmentation_to_video(
                segmentation=binary_labels,
                video=segmentation_rgb,
                color=color,
                transparency=1000,
            )

    # Create movie with colored annotated instances
    y_instances_rgb = np.copy(255 * (xs[i] / xs[i].max()))
    y_instances_rgb = [
        Image.fromarray(frame).convert("RGB") for frame in y_instances_rgb
    ]

    for event_id in range(1, ys_instances[i].max() + 1):
        event_mask = ys_instances[i] == event_id

        # Create a random color for each event
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        y_instances_rgb = add_colored_segmentation_to_video(
            segmentation=event_mask,
            video=y_instances_rgb,
            color=color,
            transparency=instances_transparency,
        )

    # Create movie with colored predicted instances
    pred_instances_rgb = np.copy(255 * (xs[i] / xs[i].max()))
    pred_instances_rgb = [
        Image.fromarray(frame).convert("RGB") for frame in pred_instances_rgb
    ]

    for event_id in range(1, pred_instances.max() + 1):
        event_mask = pred_instances == event_id

        # Create a random color for each event
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        pred_instances_rgb = add_colored_segmentation_to_video(
            segmentation=event_mask,
            video=pred_instances_rgb,
            color=color,
            transparency=instances_transparency,
        )

    # Concatenate movies vertically adding some space in between
    movie_duration = len(movie_rgb)
    space = 10

    stacked_video = []
    for frame in range(movie_duration):
        result = Image.new(
            "RGB",
            (
                movie_rgb[0].width,
                4 * movie_rgb[0].height + 4 * space + 4 * 32,
            ),
            (255, 255, 255),
        )
        y_offset = 0

        # Add text to the top of the frame
        font = ImageFont.truetype("arial.ttf", 24)
        draw = ImageDraw.Draw(result)
        draw.text((10, y_offset), "Original movie", fill=(0, 0, 0), font=font)
        y_offset += 32

        # Add the original movie frame
        result.paste(movie_rgb[frame], (0, y_offset))
        y_offset += movie_rgb[frame].size[1] + space

        # Add text above the annotated instances mask
        draw.text((10, y_offset), "Annotated instances", fill=(0, 0, 0), font=font)
        y_offset += 32

        # Add the annotated instances mask
        result.paste(y_instances_rgb[frame], (0, y_offset))
        y_offset += y_instances_rgb[frame].size[1] + space

        # Add text above the annotated segmentation mask
        draw.text(
            (10, y_offset),
            "Annotated vs. predicted segmentation masks",
            fill=(0, 0, 0),
            font=font,
        )
        y_offset += 32

        # Add the annotated segmentation mask
        result.paste(segmentation_rgb[frame], (0, y_offset))
        y_offset += segmentation_rgb[frame].size[1] + space

        # Add text above the predicted instances mask
        draw.text((10, y_offset), "Predicted instances", fill=(0, 0, 0), font=font)
        y_offset += 32

        # Add the predicted instances mask
        result.paste(pred_instances_rgb[frame], (0, y_offset))
        y_offset += pred_instances_rgb[frame].size[1] + space

        stacked_video.append(result)

    stacked_video = np.stack([np.array(frame) for frame in stacked_video])
    movie_fn = f"{training_name}_{load_epoch:06d}_{movie_id}_stacked_colored_preds_and_labels.tif"
    imageio.volwrite(os.path.join(out_dir, movie_fn), stacked_video)